# Simple Example
This Jupyter notebook runs on Colab and shows a simple example of Tooling selection using Top-K and Double Round Robin

## Install Ollama

Before we get started with Mellea, we download, install and serve ollama. We define set_css to wrap Colab output.

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &

from IPython.display import HTML, display


def set_css():
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))


get_ipython().events.register("pre_run_cell", set_css)

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


## Install Mellea
We run `uv pip install mellea` to install Mellea.

In [3]:
!uv pip install mellea -q

### **Import top_k and double_round_robin libraries**

In [4]:
!git clone https://github.com/generative-computing/mellea-contribs.git

Cloning into 'mellea-contribs'...
remote: Enumerating objects: 219, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 219 (delta 86), reused 108 (delta 44), pack-reused 58 (from 1)
Receiving objects: 100% (219/219), 351.40 KiB | 2.77 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [5]:
%cd mellea-contribs
!pip install -e .

/content/mellea-contribs
Obtaining file:///content/mellea-contribs
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mellea-contribs (pyproject.toml) ... done
  Created wheel for mellea-contribs: filename=mellea_contribs-0.1.2-py3-none-any.whl size=5868 sha256=f180b98b8dd3142068c245b8e44edaf166b082c48d3724df2c0825892be00eec
  Stored in directory: /tmp/pip-ephem-wheel-cache-eap6g_i9/wheels/4e/73/36/57f081d1b5b5a73a20d0f5515fda442a6c206e7645b5a97df0
Successfully built mellea-contribs


## **Defining sample set of tools**

In [6]:
from mellea import start_session
import mellea

m = start_session()

TOOLS = [
    {
        "name": "calendar",
        "description": "Internal service for scheduling meetings and managing events"
    },
    {
        "name": "calculator",
        "description": "Internal utility for budget and cost calculations"
    },
    {
        "name": "search",
        "description": "Internal indexed search over company documents"
    },
]

USER_QUERY = "Schedule a meeting for tomorrow and calculate the event budget"

/usr/local/lib/python3.12/dist-packages/granite_common/granite3/granite33/output.py:184: SyntaxWarning: invalid escape sequence '\d'
  f'{re.escape(CITE_START)}{{"document_id": "(\d+)"}}{re.escape(CITE_END)}'
pulling 6c02683809a8 sha256:6c026: 100%|██████████| 2.10G/2.10G [00:11<00:00, 152MB/s]
pulling 0f6ec9740c76 sha256:0f6ec:   0%|          | 0.00/7.08k [00:00<?, ?B/s]
pulling 0f6ec9740c76 sha256:0f6ec: 100%|██████████| 7.08k/7.08k [00:00<00:00, 17.1kB/s]

pulling cfc7749b96f6 sha256:cfc77:   0%|          | 0.00/11.4k [00:00<?, ?B/s]

pulling cfc7749b96f6 sha256:cfc77: 100%|██████████| 11.4k/11.4k [00:00<00:00, 27.3kB/s]


pulling ba32b08db168 sha256:ba32b:   0%|          | 0.00/417 [00:00<?, ?B/s]


                                                                                      
                                                                                       

                                                                                       


                     

**Step 1: Top-K Tool Shortlisting**

In [7]:
from mellea_contribs.tools.top_k import top_k

def shortlist_tools(
    m: mellea.MelleaSession,
    user_query: str,
    tools: list[dict],
    k: int,
):
    shortlisted = top_k(
        items=tools,
        comparison_prompt=f"""
        Select the tools most relevant to the user query:
        '{user_query}'
        """,
        m=m,
        k=k,
    )

    return shortlisted


top_k_tools = shortlist_tools(
    m,
    user_query=USER_QUERY,
    tools=TOOLS,
    k=2,
)

print("Top-K shortlisted tools:")
for tool in top_k_tools:
    print("-", tool["name"])


  0%|          | 0/2 [00:00<?, ?it/s]

=== 22:06:39-INFO ======
SUCCESS


INFO:fancy_logger:SUCCESS
  0%|          | 0/2 [00:54<?, ?it/s]

Top-K shortlisted tools:
- calendar
- calculator


**Step 2: High-Confidence Ranking with Double Round Robin**

In [8]:
from mellea_contribs.tools.double_round_robin import double_round_robin

def rank_tools_with_drr(
    m: mellea.MelleaSession,
    user_query: str,
    tools: list[dict],
):
    rankings = double_round_robin(
        items=tools,
        comparison_prompt=f"""
        Given the user query:
        '{user_query}',
        select which tool is more useful.
        """,
        m=m,
    )

    return rankings


final_rankings = rank_tools_with_drr(
    m,
    user_query=USER_QUERY,
    tools=top_k_tools,
)

print("\nFinal tool ranking:")
for tool, score in final_rankings:
    print(f"{tool['name']}: {score}")


  0%|          | 0/2 [00:00<?, ?it/s]

=== 22:07:11-INFO ======
SUCCESS


INFO:fancy_logger:SUCCESS
  0%|          | 0/2 [00:00<?, ?it/s]

=== 22:07:37-INFO ======
SUCCESS


INFO:fancy_logger:SUCCESS
  0%|          | 0/2 [00:25<?, ?it/s]


Final tool ranking:
calendar: 2
calculator: 0


**Final Tool selection**

In [10]:
best_tool = max(final_rankings, key=lambda x: x[1])[0]
print("\nSelected tool:", best_tool["name"])



Selected tool: calendar
